In [ ]:
import talos as ta

from sklearn.model_selection import GridSearchCV
import pprint
pp = pprint.PrettyPrinter(indent = 4)
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import *
from keras.wrappers.scikit_learn import KerasClassifier
from keras.datasets import mnist

import os
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

import tensorflow as tf

import keras
from keras.layers import *
from keras.callbacks import *
from keras.models import Model, Sequential
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import LeakyReLU

In [ ]:
target = pd.read_csv('target.csv')
data = pd.read_csv('data.csv')
test = pd.read_csv('test.csv')
vdata = pd.read_csv('vdata.csv')
vtarget = pd.read_csv('vtarget.csv')
wdata = pd.read_csv('whole_data.csv', header=None)
wtarget = pd.read_csv('whole_target.csv', header=None)

winedata = pd.read_csv('winedata.csv', header=None)

ccdata = pd.read_csv('creditcard.csv', header=None)

irisdata = pd.read_csv('irisdata.csv', header=None)
iristarget = pd.read_csv('iristarget.csv', header=None)

print('Data is ready !!')

In [ ]:
decode_dic = {0: 'fine_concrete',
              1: 'concrete',
              2: 'soft_tiles',
              3: 'tiled',
              4: 'soft_pvc',
              5: 'hard_tiles_large_space',
              6: 'carpet',
              7: 'hard_tiles',
              8: 'wood'}

In [ ]:
data = data.values
vdata = vdata.values
test = test.values
target = target['surface']
vtarget = vtarget['surface']

In [ ]:
print(target.shape)
print(vtarget.shape)
print(btarget.shape)
print(bvtarget.shape)
print(vdata.shape)
print(data.shape)
test.shape

In [ ]:

# data = data.reshape((2804, 128, 23))
# vdata = vdata.reshape((1006, 128, 23))
# test = test.reshape((3816, 128, 23))

data = data.reshape((358912, 23))
vdata = vdata.reshape((128768, 23))
test = test.reshape((488448, 23))

In [ ]:
model = Sequential()

# data = data.reshape((2804, 128, 23))
# vdata = vdata.reshape((1006, 128, 23))
# test = test.reshape((3816, 128, 23))
num_classes = 9


# model.add(Conv1D(16, (3,), padding='same', input_shape=data.shape[1:], use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(Conv1D(16, (3,), use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(MaxPooling1D(pool_size=(2,)))
# # model.add(SpatialDropout1D(0.25))

# model.add(Conv1D(64, (3,), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(Conv1D(64, (3,), use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(MaxPooling1D(pool_size=(2,)))
# # model.add(SpatialDropout1D(0.25))


# model.add(Flatten())
# model.add(LSTM(128, use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
model.add(Dense(64, input_dim=23, use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.3))
model.add(Dense(64, use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.3))
model.add(Dense(64, use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
# Let's train the model using RMSprop
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# loss='mearn_squared_error'
# 'adam'
# 'Nadam'
# loss = kullback_leibler_divergence
# 

In [ ]:
model.fit(data, btarget,
              batch_size=128,
              epochs=100,
              validation_data=(vdata, bvtarget),
              shuffle=True)

In [ ]:
preds = model.predict_classes(test)
preds = preds[0:-1:128]
sub['surface'] = preds
sub['surface'] = sub['surface'].map(decode_dic)
sub.head(15)

In [ ]:
p = {'activation':['tanh', 'relu'],
         'last_activation': ['softmax'],
         'optimizer': ['Nadam', 'Adam', 'RMSprop'],
         'losses': ['sparse_categorical_crossentropy'],
         'first_neuron':[8, 16, 32, 64, 128],
         'hidden_layers':[0, 1, 2],
         'kernel_initializer': ['uniform','normal'],
         'batch_size': [10,20,30,40,60,100],
         'dropout': (0, 0.5, 10),
         'shapes':['brick','long_funnel'],
         'weight_regulizer': [None],
         'epochs': [200]}

In [ ]:

def kaggle_model(x_train, y_train, x_val, y_val, params):

    model = Sequential()
        
    model.add(Dense(params['first_neuron'], input_dim=23, activation=params['activation'], kernel_initializer=params['kernel_initializer']))
    model.add(Dropout(params['dropout']))
        
    ta.model.hidden_layers(model, params, 1)
        
    model.add(Dense(9, activation=params['last_activation'], kernel_initializer=params['kernel_initializer']))
        
    model.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['acc'])

    out = model.fit(data, btarget,
                         batch_size=params['batch_size'],
                         epochs=params['epochs'],
                         validation_data=(vdata, bvtarget),
                         verbose=0)

    return out, model

    


In [ ]:
scan_object = ta.Scan(data, btarget, model=kaggle_model, params=p, grid_downsample=0.1)

In [ ]:
scan_object

In [ ]:
def build_model(optimizer, learning_rate, activation, dropout_rate,
                initilizer,num_unit):
    keras.backend.clear_session()
    model = Sequential()
    model.add(Dense(num_unit, kernel_initializer=initilizer,
                    activation=activation, input_shape=(784,)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_unit, kernel_initializer=initilizer,
                    activation=activation))
    model.add(Dropout(dropout_rate)) 
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer(lr=learning_rate),
                  metrics=['accuracy'])
    return model

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
# [:1] is for testing

batch_size = [20, 50, 100][:1]

epochs = [1, 20, 50][:1]

initilizer = ['lecun_uniform', 'normal', 'he_normal', 'he_uniform'][:1]

learning_rate = [0.1, 0.001, 0.02][:1]

dropout_rate = [0.3, 0.2, 0.8][:1]

num_unit = [10, 5][:1]

activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'][:1]

optimizer = [SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam][:1]

In [ ]:
# Creat the wrapper and pass params to GridSearchCV 
# parameters is a dict with all values

parameters = dict(batch_size = batch_size,
                  epochs = epochs,
                  dropout_rate = dropout_rate,
                  num_unit = num_unit,
                  initilizer = initilizer,
                  learning_rate = learning_rate,
                  activation = activation,
                  optimizer = optimizer)

model = KerasClassifier(build_fn=build_model, verbose=0)

models = GridSearchCV(estimator = model, param_grid=parameters, n_jobs=1)

In [ ]:
best_model = models.fit(x_train, y_train)
print('Best model :')
pp.pprint(best_model.best_params_)